## Lenet-5

In [67]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
import numpy as np

In [70]:
img_height = 28
img_width = 28
img_channels=1
input_shape = (img_height, img_width, img_channels)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
num_classes = 10
print('before',x_train.shape)
x_train = x_train[:, :, :, np.newaxis] # 차원추가
x_test = x_test[:, :, :, np.newaxis]
print('after',x_train.shape)

x_train,x_test=x_train/255.0,x_test/255.0


before (60000, 28, 28)
after (60000, 28, 28, 1)


In [71]:
model=Sequential()
model.add(Conv2D(6,kernel_size=(5,5),padding='same',activation='relu',input_shape=(img_height,img_width,img_channels)))
# model.add(Conv2D(6,kernel_size=(5,5),padding='same',activation='relu',input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16,kernel_size=(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(120,activation='relu'))
model.add(Dense(84,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
# model.add(Dense(10,activation='softmax'))

### Class

In [87]:
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout

In [73]:
class LeNet(Model):
    def __init__(self,num_classes):
        super(LeNet5,self).__init__()
        self.conv1=Conv2D(6,kernel_size=(5,5),padding='same',activation='relu')
        self.conv2=Conv2D(16,kernel_size=(5,5),activation='relu')
        self.max_pool=MaxPooling2D(pool_size=(2,2))
        self.flatten=Flatten()
        self.dense1=Dense(120,activation='relu')
        self.dense2=Dense(84,activation='relu')
        self.dense3=Dense(num_classes,activation='softmax')
    def call(self,x):
        x=self.max_pool(self.conv1(x))
        x=self.max_pool(self.conv1(x))
        x=self.flatten(x)
        x=self.dense3(self.dense2(self.dense1(x)))
        return x

In [74]:
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [75]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 28, 28, 6)         156       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 120)               48120     
_________________________________________________________________
dense_25 (Dense)             (None, 84)              

In [76]:
callbacks=[tf.keras.callbacks.EarlyStopping(patience=3,monitor='val_loss'),
          tf.keras.callbacks.TensorBoard(log_dir='./logs',histogram_freq=1)]
model.fit(x_train,y_train,batch_size=32,epochs=80,validation_data=(x_test,y_test),callbacks=callbacks)

Epoch 1/80
1875/1875 [==============================] - 12s 4ms/step - loss: 0.4699 - accuracy: 0.8599 - val_loss: 0.1670 - val_accuracy: 0.9468
Epoch 2/80
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1246 - accuracy: 0.9616 - val_loss: 0.0808 - val_accuracy: 0.9740
Epoch 3/80
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0903 - accuracy: 0.9722 - val_loss: 0.0769 - val_accuracy: 0.9743
Epoch 4/80
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0743 - accuracy: 0.9770 - val_loss: 0.0665 - val_accuracy: 0.9793
Epoch 5/80
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0626 - accuracy: 0.9801 - val_loss: 0.0691 - val_accuracy: 0.9768
Epoch 6/80
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0553 - accuracy: 0.9825 - val_loss: 0.0697 - val_accuracy: 0.9774
Epoch 7/80
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0504 - accuracy: 0.9847 - val_loss: 0.0691 - val_accuracy

In [80]:
%tensorboard --logdir ./logs

## regularization

In [81]:
l1_reg=tf.keras.regularizers.l1(0.01)
model=Sequential()
model.add(Conv2D(6,kernel_size=(5,5),padding='same',activation='relu',input_shape=(img_height,img_width,img_channels),
                kernel_regularizer=l1_reg))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16,kernel_size=(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(120,activation='relu'))
model.add(Dense(84,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))


In [83]:
model.compile(optimizer='nadam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 28, 28, 6)         156       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 120)               48120     
_________________________________________________________________
dense_28 (Dense)             (None, 84)              

In [84]:
callbacks=[tf.keras.callbacks.EarlyStopping(patience=3,monitor='val_loss'),
          tf.keras.callbacks.TensorBoard(log_dir='./logs_03',histogram_freq=1)]
model.fit(x_train,y_train,batch_size=32,epochs=80,validation_data=(x_test,y_test),callbacks=callbacks)

Epoch 1/80
1875/1875 [==============================] - 19s 10ms/step - loss: 0.2463 - accuracy: 0.9425 - val_loss: 0.1102 - val_accuracy: 0.9754
Epoch 2/80
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0970 - accuracy: 0.9781 - val_loss: 0.0830 - val_accuracy: 0.9802
Epoch 3/80
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0730 - accuracy: 0.9829 - val_loss: 0.0652 - val_accuracy: 0.9852
Epoch 4/80
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0597 - accuracy: 0.9862 - val_loss: 0.0554 - val_accuracy: 0.9865
Epoch 5/80
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0508 - accuracy: 0.9883 - val_loss: 0.0619 - val_accuracy: 0.9840
Epoch 6/80
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0453 - accuracy: 0.9895 - val_loss: 0.0428 - val_accuracy: 0.9896
Epoch 7/80
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0397 - accuracy: 0.9912 - val_loss: 0.0470 - val_

In [85]:
%tensorboard --logdir ./logs_03

## Dropout

In [88]:
l1_reg=tf.keras.regularizers.l1(0.01)
model=Sequential()
model.add(Conv2D(6,kernel_size=(5,5),padding='same',activation='relu',input_shape=(img_height,img_width,img_channels),
                kernel_regularizer=l1_reg))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16,kernel_size=(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(120,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(84,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes,activation='softmax'))

In [89]:
model.compile(optimizer='nadam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 28, 28, 6)         156       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 120)               48120     
_________________________________________________________________
dropout (Dropout)            (None, 120)             

In [90]:
callbacks=[tf.keras.callbacks.EarlyStopping(patience=3,monitor='val_loss'),
          tf.keras.callbacks.TensorBoard(log_dir='./logs_03_02',histogram_freq=1)]
model.fit(x_train,y_train,batch_size=32,epochs=80,validation_data=(x_test,y_test),callbacks=callbacks)

Epoch 1/80
1875/1875 [==============================] - 19s 10ms/step - loss: 0.3218 - accuracy: 0.9247 - val_loss: 0.1085 - val_accuracy: 0.9784
Epoch 2/80
1875/1875 [==============================] - 18s 10ms/step - loss: 0.1256 - accuracy: 0.9718 - val_loss: 0.0840 - val_accuracy: 0.9813
Epoch 3/80
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0976 - accuracy: 0.9780 - val_loss: 0.0642 - val_accuracy: 0.9860
Epoch 4/80
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0802 - accuracy: 0.9823 - val_loss: 0.0598 - val_accuracy: 0.9865
Epoch 5/80
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0707 - accuracy: 0.9841 - val_loss: 0.0550 - val_accuracy: 0.9871
Epoch 6/80
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0646 - accuracy: 0.9850 - val_loss: 0.0538 - val_accuracy: 0.9867
Epoch 7/80
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0587 - accuracy: 0.9867 - val_loss: 0.0603 -

In [91]:
%tensorboard --logdir ./logs_03_02